In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
# from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, InputLayer
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
# from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, normalize
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
tf.__version__

In [ ]:
train_df = pd.read_csv(os.path.join('data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('data', 'unionTest.csv'))

In [ ]:
X_train, y_train = train_df.drop(['Severity', 'sequence_id'], axis=1), train_df['Severity']
X_test, y_test = test_df.drop(['Severity', 'sequence_id'], axis=1), test_df['Severity']

In [ ]:
# scaler_name = 'RobustScaler.pkl'
scaler_name = 'MinMaxScaler.pkl'
scaler = joblib.load(os.path.join('saved_scaler', scaler_name))

In [ ]:
model_name = 'trained_at_2022-10-19_13-58-43_using_MinMaxScaler.h5'
model = load_model(os.path.join('saved_models', model_name))
model.summary()

In [ ]:
X_test_scaled = scaler.transform(X_test)
X_train_scaled = scaler.transform(X_train)
def modify_X(x):
    result = x.copy()
    result = result.reshape(-1, 21350//25, 50)
    return result
def modify_y(df):
    result = pd.get_dummies(df)
    result = result.to_numpy()
    return result

X_train_modified = modify_X(X_train_scaled)
y_train_modified = modify_y(y_train)
X_test_modified = modify_X(X_test_scaled)
y_test_modified = modify_y(y_test)

In [ ]:
len(X_test_modified), len(y_test_modified)

In [ ]:
train_predict = model.predict(X_train_modified)
train_real_predict = np.argmax(train_predict, axis=1)+1
# for i in range(len(y_train)):
#     print(f"Index:{i}, Predict:{train_real_predict[i]}, Real:{y_train[i]}")

In [ ]:
f1_train = f1_score(y_train, train_real_predict)
accuracy_train = accuracy_score(y_train, train_real_predict)
print(f"f1: {f1_train}\naccuracy: {accuracy_train}")
print(classification_report(y_train, train_real_predict))
print("----------------------------------------------")
sns.heatmap(confusion_matrix(y_train, train_real_predict),annot = True,fmt = '2.0f')
print()

In [ ]:
test_predict = model.predict(X_test_modified)
test_real_predict = np.argmax(test_predict, axis=1)+1
# for i in range(len(y_test)):
#     print(f"Index:{i}, Predict:{test_real_predict[i]}, Real:{y_test[i]}")

In [ ]:
f1_test = f1_score(y_test, test_real_predict)
accuracy_test = accuracy_score(y_test, test_real_predict)
print(f"f1: {f1_test}\naccuracy: {accuracy_test}")
print(classification_report(y_test, test_real_predict))
print("----------------------------------------------")
sns.heatmap(confusion_matrix(y_test, test_real_predict),annot = True,fmt = '2.0f')
print()